# 두번째/.

# **결측값 처리 하기!**
## 결측치를 처리하기 위한 방법
### 1, 결측치가 있는 변수를 삭제한다
### 2, 결측치를 다른 값으로 채워넣는다
### 3, 결측치를 채워 넣는데 좀 이상한 방법을 거친다.

## 1, A Simple Option: Drop Columns with Missing Values
    data_without_missing_values = original_data.dropna(axis=1)

#### training dataset 과 test dataset에서 같은 열을 모두 drop할 경우

    cols_with_missing = [col for col in original_data.columns 
                                 if original_data[col].isnull().any()]
    redued_original_data = original_data.drop(cols_with_missing, axis=1)
    reduced_test_data = test_data.drop(cols_with_missing, axis=1)

#### 열의 대부분 값이 누락되는 경우는 유용할 수 있음
#### 그러나 해당 열에 유용한 정보가 있는 경우 이 정보를 활용할 수 없으므로 좋은 방법은 아님

## 2, A Better Option: Imputation
#### missing value에 대표치 넣는 방법, 대부분 대표치로 평균을 활용
#### 정확한 값을 넣는 것은 아니지만 열을 삭제하는 것 보다 정확한 모델을 만들 수 있다.
    from sklearn.preprocessing import Imputer
    my_imputer = Imputer()
    data_with_imputed_values = my_imputer.fit_transform(original_data)
#### scikit-learn Pipeline( 모델 구축, 모델 검증 및 모델 배치를 단순화)에 포함될 수 있다는 장점

## 3, An Extension To Imputation
#### 1) 원본 데이터를 복사한다.(원본데이터의 변화를 막기위해서)
    new_data = original_data.copy()

####  2) 새로운 열을 만든다. (채워넣을 지표가 되는..?)make new columns indicating what will be imputed
    cols_with_missing = (col for col in new_data.columns 
                                 if new_data[c].isnull().any())
    for col in cols_with_missing:
        new_data[col + '_was_missing'] = new_data[col].isnull()

#### 3) 채워넣는다. Imputation
    my_imputer = Imputer()
    # Imputer(missing_values=’NaN’, strategy=’mean’, axis=0, verbose=0, copy=True)[source]¶
    new_data = my_imputer.fit_transform(new_data)
    
### 3번 방법은 어떤 경우 굉장히 효과적이지만 그 외에는 전혀 효과가 없다.. 어떤 경우가 뭔지는 모르겠다..

In [4]:
import pandas as pd
melb_data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

melb_target = melb_data.Price
melb_predictors = melb_data.drop(['Price'], axis=1)
# axis=0 이면 열을 삭제하고 , =1 이면 행을 삭제한다.

melb_numeric_predictors = melb_predictors.select_dtypes(exclude=['object'])
# 예제를 쉽게 하기 위해 숫자형 데이터만 사용한다. 

    score_dataset(X_train, X_test, y_train, y_test) 를 사용.
각각의 결측치 처리 방식이 어떤 결과를 가지고 오는지를 보여주기 위함

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(melb_numeric_predictors, 
                                                    melb_target,
                                                    train_size=0.7, 
                                                    test_size=0.3, 
                                                    random_state=0) #random_state

def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

In [6]:
cols_with_missing = [col for col in X_train.columns 
                                 if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test  = X_test.drop(cols_with_missing, axis=1)

print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

In [7]:
from sklearn.preprocessing import Imputer

my_imputer = Imputer()
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(X_test)
print("Mean Absolute Error from Imputation:")
print(score_dataset(imputed_X_train, imputed_X_test, y_train, y_test))

In [8]:
imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()

cols_with_missing = (col for col in X_train.columns 
                                 if X_train[col].isnull().any())

for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

# Imputation
my_imputer = Imputer()
imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)

print("Mean Absolute Error from Imputation while Track What Was Imputed:")
print(score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_test))

# **범주형 데이터 다루기!**
### 범주형 데이터는 제한된 값들만을 가진 데이터 입니다.
#### ex. 소유한 자동차의 브랜드
### 이러한 범주형 데이터를 인코딩 하지 않고 학습을 시키면 오류가 발생하기 때문에 인코딩이 필요합니다~

## One-Hot Encoding
#### 가장 표준화된 방법. 범주형 데이터의 변수가 많지 않은 경우에 사용 ( 15개 이하)
#### 원본 데이터가 의미하는 내용을 새로운 2진 열(0,1)로 만들어서 인코딩함.
![one-hot 예시](https://i.imgur.com/WAOnNSI.png)

In [9]:
# Read the data
import pandas as pd
train_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# Drop houses where the target is missing
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

target = train_data.SalePrice

# Since missing values isn't the focus of this tutorial, we use the simplest
# possible approach, which drops these columns. 
# For more detail (and a better approach) to missing values, see
# https://www.kaggle.com/dansbecker/handling-missing-values
cols_with_missing = [col for col in train_data.columns 
                                 if train_data[col].isnull().any()]                                  
candidate_train_predictors = train_data.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)
candidate_test_predictors = test_data.drop(['Id'] + cols_with_missing, axis=1)

# "cardinality" means the number of unique values in a column.
# We use it as our only way to select categorical columns here. This is convenient, though
# a little arbitrary.
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

In [10]:
train_predictors.dtypes.sample(10)
# Pandas assigns a data type (called a dtype) to each column or Series. Let's see a random sample of dtypes from our prediction data:

In [11]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)

### pd.get_dummies() 사용
### 선형 회귀를 수행하고 범주형 변수를 인코딩 할 때 완벽한 공선성이 문제가 될 수 있다.
### 이 문제를 해결하려면 n-1 개의 열을 사용하는 것이 좋다.
#### 몰라 일단 범주형 변수를 인코딩 할 때 get_dummies() 함수를 쓰자...


## 인코딩 하는거랑 범주형 변수를 drop 하는 것 비교하기
#### 아래의 경우에는 별반 차이가 없다..

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])

mae_without_categoricals = get_mae(predictors_without_categoricals, target)

mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, target)

print('Mean Absolute Error when Dropping Categoricals: ' + str(int(mae_without_categoricals)))
print('Mean Abslute Error with One-Hot Encoding: ' + str(int(mae_one_hot_encoded)))

# 여러 파일에 적용하기.
#### 방금까지는 train_data 를 한번 인코딩 했음. 여러 파일(test_data set or some other data that you'd like to make predictions for) 이 있으면 어케해야 할까?
### sklearn 은 열 순서에 민감하므로 각각의 파일들을 정렬해놓지 않으면 결과가 제대로 안나옴.

## 정렬하는 명령을 사용하여 동일한 방식으로 인코딩 되었는지 확인하기

In [13]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)
final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='left', 
                                                                    axis=1)

### align 명령은 두 데이터 세트에서 동일한 순서로 열이 표시되도록 한다.
    ### 이 열 이름을 보고 어떤식으로 정렬이 되어있는지, 잘 되어있는지를 파악한다.

### 인수 join = 'left'는 SQL의 'left join' 과 동일한 작업을 수행하도록 지정함 
    한 데이터 집합에 표시되는 열이 있고 다른 데이터 집합에 표시되는 열이 없으면, 우리는 우리의 교육 데이터에서 열을 기준으로 열들을 정확하게 유지

# Partial Dependence Plots
### 부분 의존도 그래프는 각 변수 또는 예측 변수가 모델의 예측에 미치는 영향을 보여줍니다.
### 다음과 같은 질문에서 유용하다.
    - 남성과 여성의 임금 격차는 교육 배경이나 직장 경험의 차이와는 달리 성별에만 어느 정도 영향을 미칩니까?

    - 주택 특성에 대한 통제, 경도 및 위도가 주택 가격에 미치는 영향은 무엇입니까? 

    - 두 그룹간에 건강상의 차이점은 다이어트의 차이 또는 다른 요인들 때문입니까?

In [14]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.preprocessing import Imputer

cols_to_use = ['Distance', 'Landsize', 'BuildingArea']

def get_some_data():
    data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')
    y = data.Price
    X = data[cols_to_use]
    my_imputer = Imputer()
    imputed_X = my_imputer.fit_transform(X)
    return imputed_X, y
    
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

# get_some_data is defined in hidden cell above.
X, y = get_some_data()
# scikit-learn originally implemented partial dependence plots only for Gradient Boosting models
# this was due to an implementation detail, and a future release will support all model types.
my_model = GradientBoostingRegressor()
# fit the model as usual
my_model.fit(X, y)
# Here we make the plot
my_plots = plot_partial_dependence(my_model,       
                                   features=[0, 2], # column numbers of plots we want to show
                                   X=X,            # raw predictors data.
                                   feature_names=['Distance', 'Landsize', 'BuildingArea'], # labels on graphs
                                   grid_resolution=10) # number of values to plot on x axis

### 보면 우리가 구하고자 하는 Price 와 변수 Distance, BuildingArea 와의 관계를 볼 수 있다.

![타이타닉 예시](https://i.imgur.com/mJex49E.png)

### Partial dependence plots 은 복잡한 모델에서 통찰력을 얻을 수 있다.
### 또한 친구들이나 비전공자들에게도 쉽게 설명해줄 수 있다.

### 하지만 인과관계를 전혀 알 수 없다는 단점도 존재한다.

# **Pipeline**
http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

#### fit and transform method
#### 중간중간에 transform 및 fit 을 하고, 마지막 사용자는 적합성(fit)만 구현하면 됨.
#### memory argument 를 이용하면 cach 를 할 수 있다.


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read Data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]
y = data.Price
train_X, test_X, train_y, test_y = train_test_split(X, y)

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer

my_pipeline = make_pipeline(Imputer(), RandomForestRegressor())

In [17]:
my_pipeline.fit(train_X, train_y)
predictions = my_pipeline.predict(test_X)

In [18]:
# 비교를 위해 pipeline 이 없는 것도 만든다.
my_imputer = Imputer()
my_model = RandomForestRegressor()

imputed_train_X = my_imputer.fit_transform(train_X)
imputed_test_X = my_imputer.transform(test_X)
my_model.fit(imputed_train_X, train_y)
predictions = my_model.predict(imputed_test_X)

### Pipeline 을 만드는건 간단한 코드로 만들 수 있지만, 유용성은 모델이 복잡해질수록 높아진다.

# **Cross-Validation**
### Cross-Validation and Train-Test Split
#### 데이터의 수가 적으면 시간이 오래 걸리더라도 Cross-Validation 을 사용하는게 좋다.
#### 데이터의 수가 적다를 구분하는 기준은 없지만, 보통 모델을 실행하는 데 몇 분정도 시간이 걸린다면 Cross-Validation 으로 전환하는게 좋다.
#### 아니면 둘 다 해보고 성능이 비슷하다면, 시간이 덜 걸리는 Train-Test Split 을 사용하는게 낫다.

![교차검증](https://i.imgur.com/pgVmXf7.png)

In [19]:
import pandas as pd
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]
y = data.Price

### Cross-Validation 을 할 때 Pipeline 을 안만들면 넘나리 어렵기 떄문에 만든다.

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
my_pipeline = make_pipeline(Imputer(), RandomForestRegressor())

### 점수는요~

In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(my_pipeline, X, y, scoring='neg_mean_absolute_error')
print(scores)

### 여기서 음의 MAE 를 이용하는게 특이한데, 이렇게 하면  해당 관례와 일관성을 유지할 수 있습니다....;;
### 무슨말인가 하면, sklearn 에서의 convention 은 수치가 높을수록 좋은것인데, 여기서 음수로 돌려버리면 그 convention 과의 일관성이 유지된다는 말인듯

In [22]:
print('Mean Absolute Error %2f' %(-1 * scores.mean()))

## 그래서 결론은 Cross-Validation 을 이용하면 코드 퀄리티가 높아지고 정확도도 높아져서 좋다!

# **Data Leakage**
https://blog.naver.com/tjdudwo93/221085844907 -- 이거 보는걸 추천
### 이 부분이 존재하면, 결과를 내기 전까진 데이터 모델이 정확해 보이지만 사실상 부정확한 모델이 구성된다.
    1, Leaky Predict
    2, Leaky Validation Stragies

## Leaky Predict.
#### 예측을 할 수 없는 데이터를 예측자가 예측하려고 할 때 발생..
#### 예
![폐암](https://i.imgur.com/PpEdi63.png)
#### 해결방법
![해결방법](https://i.imgur.com/Z5le4tI.png)


##  Leaky Validation Stragies
#### Train_data 와 test_data 를 나눌 때 많이 발생함.
#### 해결방법으로는 그 데이터에 대한 지식을 가지고 있으면 됨.
#### 보편적인 해결방법으로는 유효성 검사로 simple train_test_split 를 하는 경우에는 Validation_data 에 대해서는 모든 처리과정을 제외함
#### Pipeline 을 이용하면 보다 쉽게 가능함.


## 예시

In [23]:
import pandas as pd

data = pd.read_csv('../input/aer-credit-card-data/AER_credit_card_data.csv', 
                   true_values = ['yes'],
                   false_values = ['no'])
print(data.head())

### 데이터가 적어서 Cross-Validation 함

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

y = data.card
X = data.drop(['card'], axis=1)

# Since there was no preprocessing, we didn't need a pipeline here. Used anyway as best practice
modeling_pipeline = make_pipeline(RandomForestClassifier())
cv_scores = cross_val_score(modeling_pipeline, X, y, scoring='accuracy')
print("Cross-val accuracy: %f" %cv_scores.mean())

#### 정확한 98 %의 모델을 찾는 것은 매우 드뭅니다. 그런 일이 발생하지만 데이터 유출 여부를 면밀히 조사해야 함.

In [25]:
expenditures_cardholders = data.expenditure[data.card]
expenditures_noncardholders = data.expenditure[~data.card]

print('Fraction of those who received a card with no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))
print('Fraction of those who received a card with no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))

In [26]:
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)
cv_scores = cross_val_score(modeling_pipeline, X2, y, scoring='accuracy')
print("Cross-val accuracy: %f" %cv_scores.mean())